In [ ]:
#Imports
import numpy as np
import pandas as pd
import itertools
import sys
import warnings
import numpy as np
import pandas as pd
from sklearn.calibration import LinearSVC
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import r2_score


# Different feature selection methods
import sklearn.feature_selection as fs

In [ ]:
# Input SCF Data and Coordinates:
data = pd.read_csv('data/SCF_data.csv')

# Input elements | Ex. types_elements_init = ['Sr','Ti', 'O', 'He']
types_elements_init = []

# Input grid increment | Ex. grid_increment = 1 for a 7 x 7, 49 grid point system
grid_inc = 0

# Input amount of grid points, should be a square | Ex. grid_points = 49
grid_points = 0

# Be sure to input top_features (will be prompted later)
top_features = []

In [ ]:
# Independent variable initialization
features = data.iloc[:, 4:]
SCFshift = data.iloc[:, 3:4]
xy = data.iloc[:, :2]
topFeatures = data[top_features]

## Feature Generation

In [ ]:
def fc(rij, r_cut, type_func):
    if type_func == 'cos':
        if rij <= r_cut:
            return 0.5*(np.cos(np.pi*rij/r_cut) + 1.0)
        elif rij > r_cut:
            return 0.0
    elif type_func == 'tanh':
        if rij <= r_cut:
            return np.tanh(1-rij/r_cut)**3.0
        elif rij > r_cut:
            return 0.0


def get_eta_values(rcut, n, min_val, max_val):
    eta = np.zeros((2*(n+1)), dtype=float)
    rs = np.zeros((2*(n+1)), dtype=float)
    ran = max_val - min_val
    def R_sm(i, m): return rcut/(i**(m/i))
    for i in range(n + 1):
        # eta[i] = 2/( (i+1)*ran/( n + 1 ) )**2.0
        eta[i] = (1.0/R_sm(n + 1, i))**2.0
        rs[i] = 0.0
    for i in range(n + 1):
        rs[n+i+1] = R_sm(n + 1, i)
        eta[2*(n+1)-i-1] = 1.0/(R_sm(n + 1, n+1-i) - R_sm(n + 1, n-i))**2.0
    return eta, rs


def G1(distances, r_cut, type_func, types_elements, elements):
    nvirtual = elements.count('He')
    nreal = len(elements)-nvirtual
    g1desc = np.zeros((nvirtual, len(types_elements)-1), dtype=float)
    for i, typei in enumerate(elements):
        if typei == 'He':  # only compute G1 for virtual atoms
            for j, typej in enumerate(elements):
                if typej != 'He':  # only compute G1 wrt real atoms
                    rij = distances[i, j]
                    idx = types_elements.index(elements[j])
                    g1desc[i-nreal][idx] += fc(rij, r_cut, type_func)
    return g1desc
# For G2 and G3, each row has the ith atoms descriptors
# and each column are the descriptors for different eta/kappa values
#            eta1       eta2     ...
# atom1  [ desc1,1    desc1,2     ... ]
# atom2  [ desc2,1    desc2,2     ... ]


def G2(distances, r_cut, eta, rs, type_func, types_elements, elements):
    nvirtual = elements.count('He')
    # Dummy atoms were appended to the end of the list; nreal shows the length of real atoms when deleting the # of grid points from the count
    nreal = len(elements)-nvirtual
    # len(types) -1 represents the dimension of the real atoms only (there are two atom types here, real and dummies)
    g2desc = np.zeros((nvirtual, len(eta), len(types_elements)-1), dtype=float)
   # print(nreal,nvirtual)
    eta = np.array(eta)
    for i, typei in enumerate(elements):
        if typei == 'He':
            for j, typej in enumerate(elements):
                if typej != 'He':  # only compute G1 with respect to real atoms;
                    rij = distances[i, j]
                    f_fun = fc(rij, r_cut, type_func)
                   # rs_fac = np.multiply( -1, np.square( np.subtract( rij, rs ) ) )
                    # np.multiply( eta, rs_fac ) )
                    exp_fac = np.exp(-eta*(rij-rs)**2)
                    idx = types_elements.index(elements[j])
                    g2desc[i-nreal, :, idx] += exp_fac * \
                        f_fun  # np.multiply(exp_fac, f_fun)

    return g2desc


def G3(distances, r_cut, kappa, type_func, types_elements, elements):
    nvirtual = elements.count('He')
    g3desc = np.zeros(
        (nvirtual, len(kappa), len(types_elements)-1), dtype=float)
    kappa = np.array(kappa)

    # Dummy atoms were appended to the end of the list; nreal shows the length of real atoms when deleting the # of grid points from the count
    nreal = len(elements)-nvirtual
    # len(types) -1 represents the dimension of the real atoms only (there are two atom types here, real and dummies)
    g3desc = np.zeros(
        (nvirtual, len(kappa), len(types_elements)-1), dtype=float)
    # print(nreal,nvirtual)

    for i, typei in enumerate(elements):
        if typei == 'He':
            for j, typej in enumerate(elements):
                if typej != 'He':  # only compute G1 with respect to real atoms;
                    rij = distances[i, j]
                    f_fun = fc(rij, r_cut, type_func)
                    # rs_fac = np.multiply( -1, np.square( np.subtract( rij, rs ) ) )
                    # np.multiply( eta, rs_fac ) )
                    cos_fac = np.cos(kappa * rij)
                    idx = types_elements.index(elements[j])
                    g3desc[i-nreal, :, idx] += cos_fac * \
                        f_fun  # np.multiply(exp_fac, f_fun)

    return g3desc
# Lambda can have values of +1 or -1
# zeta can have integer values greater than zero


def G4(distance_vectors, distances, r_cut, type_func, elements, combo_types):
    # init_size = list( np.shape( bp_calcs ) )
    # init_size.append( len( type_combos ) )
    nvirtual = elements.count('He')
    ncombos = len(combo_types)
    g4desc = np.zeros((nvirtual, len(zeta), ncombos), dtype=float)
    # combos = np.array( combos )
    lambd = np.array([1.0])
    for i, typei in enumerate(elements):
        if typei == 'He':
            for j, typej in enumerate(elements):
                if typej != 'He':

                    rij = distances[i, j]
                    rij_vec = distance_vectors[i, j]

                    for k, typek in enumerate(elements):
                        if typek != 'He':
                            if j == k:
                                continue
                            rik = distances[i, k]
                            rik_vec = distance_vectors[i, k]

                            rjk = distances[j, k]
                            rjk_vec = distance_vectors[j, k]
                            for l, m in enumerate(zeta):

                                cost = np.dot(rij_vec, rik_vec)/(rij*rik)

                                ang_lin = (2.0**(1-m))*(1+lambd*cost)**m

                                exp_fac = np.exp(-eta[0]
                                                 * (rij**2+rik**2+rjk**2))

                                fc_fac = fc(rij, r_cut, type_func)*fc(rik,
                                                                      r_cut, type_func)*fc(rjk, r_cut, type_func)

                                if [typej, typek] in combo_types:
                                    idx = combo_types.index([typej, typek])
                                else:
                                    idx = combo_types.index([typek, typej])
                                g4desc[i-nreal, l, idx] += ang_lin * \
                                    exp_fac*fc_fac
    return g4desc
# Lambda can have values of +1 or -1
# zeta can have integer values greater than zero


def G5(distance_vectors, distances, r_cut, type_func, elements, combo_types):
    # init_size = list( np.shape( bp_calcs ) )
    # init_size.append( len( type_combos ) )
    nvirtual = elements.count('He')
    ncombos = len(combo_types)
    g5desc = np.zeros((nvirtual, len(zeta), ncombos), dtype=float)
    # combos = np.array( combos )
    lambd = np.array([1.0])
    for i, typei in enumerate(elements):
        if typei == 'He':
            for j, typej in enumerate(elements):
                if typej != 'He':

                    rij = distances[i, j]
                    rij_vec = distance_vectors[i, j]

                    for k, typek in enumerate(elements):
                        if typek != 'He':
                            if j == k:
                                continue
                            rik = distances[i, k]
                            rik_vec = distance_vectors[i, k]

                            rjk = distances[j, k]
                            rjk_vec = distance_vectors[j, k]
                            for l, m in enumerate(zeta):

                                cost = np.dot(rij_vec, rik_vec)/(rij*rik)

                                ang_lin = (2.0**(1-m))*(1+lambd*cost)**m

                                exp_fac = np.exp(-eta[0]*(rij**2+rik**2))

                                fc_fac = fc(rij, r_cut, type_func) * \
                                    fc(rik, r_cut, type_func)

                                if [typej, typek] in combo_types:
                                    idx = combo_types.index([typej, typek])
                                else:
                                    idx = combo_types.index([typek, typej])
                                g5desc[i-nreal, l, idx] += ang_lin * \
                                    exp_fac*fc_fac
    return g5desc

In [ ]:
from ase.io import read
from ase.visualize import view
from ase.build import add_adsorbate
from ase.io import write

# convert data csv to xyz
ase.write('data.xyz', data, format='xyz')

total_ase_atoms = read('data.xyz')

#iterate through a 6 by 6 equally space grid of adsorption sites, with one corner at 0, 0, and one corner at 7.63363 , 7.63363. Add an adsorbate at each site as well as the opposite height of the entire slab.
# 7.63363/7 = 1.0905185714285713
# 7.63363/6 = 1.2722725

#write np code that takes the square root of 64


xx_reshape = np.zeros((64, 1))
yy_reshape = np.zeros((64, 1))
for i in range(0, np.sqrt(grid_points)):
    for j in range(0, np.sqrt(grid_points)):
        print(i*grid_inc, j*grid_inc)
        xx_reshape[i*np.sqrt(grid_points)+j] = i*grid_inc
        yy_reshape[i*np.sqrt(grid_points)+j] = j*grid_inc
        
        add_adsorbate(total_ase_atoms, 'He', 3, (i*grid_inc, j*grid_inc))
        
        # energy_total = model.get_potential_energy()

view(total_ase_atoms, viewer='x3d')

In [ ]:
distance_vectors=total_ase_atoms.get_all_distances(mic=True,vector=True)
distances=total_ase_atoms.get_all_distances(mic=True)
types_elements = types_atoms_init
elements = total_ase_atoms.get_chemical_symbols()
nvirtual=elements.count('He')
nreal=len(elements)-nvirtual

In [ ]:
type_func='cos'
r_cut = 12.0
eta,rs=get_eta_values(12,6,1,6)
kappa=[0.5, 1.0, 1.5, 2.0]
zeta=[1,2,4,8,16,32]

In [ ]:
gridpoints=pd.DataFrame(xx_reshape,columns=['X'])
gridpoints['Y']=yy_reshape

In [ ]:
g2=G2(distances,r_cut, eta, rs, type_func, types_elements, elements)
gridpoints['G2-0']=g2[:,0,0]
gridpoints['G2-1']=g2[:,1,0]
gridpoints['G2-2']=g2[:,2,0]

In [ ]:
g3=G3(distances, r_cut, kappa, type_func, types_elements, elements)
gridpoints['G3-0']=g3[:,0,0]
gridpoints['G3-1']=g3[:,1,0]
gridpoints['G3-2']=g3[:,2,0]
gridpoints['G3-3']=g3[:,3,0]

In [ ]:
ncombos = (len(types_elements) - 1)*len(types_elements)//2
combo_types = []
for i in range(len(types_elements)-1):
    for j in range(i,len(types_elements)-1):
        combo_types.append([types_elements[i],types_elements[j]])
g4=G4(distance_vectors,distances,r_cut,type_func,elements,combo_types)
gridpoints['G4-0']=g4[:,0,0]
gridpoints['G4-1']=g4[:,1,0]
gridpoints['G4-2']=g4[:,2,0]
gridpoints['G4-3']=g4[:,3,0]
gridpoints['G4-4']=g4[:,4,0]
gridpoints['G4-5']=g4[:,5,0]

In [ ]:
# gridpoints

## Feature Selection and Clustering

In [ ]:
#NORMALIZED DATA

def perform_clustering_norm(features, SCFshift, algorithm_index,n_clusters):
    # Generate combinations of variables
    variable_combinations = list(itertools.combinations(features.columns, 2))
    total_combinations = len(variable_combinations)

    # Select the algorithm based on the given index
    algorithms = [LinearSVC(), KMeans(n_clusters=n_clusters)]
    algorithm = algorithms[algorithm_index]

    # Initialize an empty list to store results
    result_list = []

    # Normalize the feature and SCFshift data
    scaler = StandardScaler()
    scaled_features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
    scaled_SCFshift = scaler.fit_transform(SCFshift)

    # Perform clustering on each combination of variables
    for i, combination in enumerate(variable_combinations):
        variables = scaled_features[list(combination)]

        # Fit the clustering algorithm to the data
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            algorithm.fit(variables)

        # Overlay clusters on scaled SCFshift data
        labels = algorithm.labels_
        score = silhouette_score(scaled_SCFshift, labels)

        # Store the results in the list
        result_list.append({'Variable 1': combination[0],
                            'Variable 2': combination[1],
                            'Silhouette Score': score})

        # Update progress
        progress = (i + 1) / total_combinations * 100
        sys.stdout.write('\r')
        sys.stdout.write(f"Clustering Progress: [{progress:.1f}%] ")
        sys.stdout.flush()

    # Convert the list to a DataFrame
    result_df = pd.DataFrame(result_list)

    # Sort the DataFrame by silhouette score in descending order
    result_df = result_df.sort_values(by='Silhouette Score', ascending=False)

    # Select the top 100 variables
    top_100_variables = result_df.head(100)

    #convert top 100 variables to csv named "top100norm.csv"
    top_100_variables.to_csv('top100norm.csv')

    return top_100_variables


# Example usage
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    top_variables = perform_clustering_norm(features, SCFshift, 1, 4)

#save top variables to a csv file named "oxygen_abs_top_var.csv"
top_variables.to_csv('oxygen_abs_top_var_4_norm.csv')

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    top_variables = perform_clustering_norm(features, SCFshift, 1, 3)

top_variables.to_csv('oxygen_abs_top_var_3_norm.csv')

print(top_variables)


### Cluster visualization

In [ ]:
# Input top features | Ex. top_features = ['G4-3  R_cut = 8', 'G5-5  R_cut = 11']
top_features_selected = []

top_feature_df = data[top_features_selected]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Create a DataFrame with the variables from top_feat
variables_df = data[top_features]

# Perform clustering
kmeans = KMeans(n_clusters=4)  # Set the desired number of clusters
kmeans.fit(variables_df)

# Assign cluster labels to each index of the SCF values
cluster_labels = kmeans.labels_

data['Cluster_labels'] = kmeans.labels_

# Obtain the SCFshift values
scf_shift_values = data['SCFshift']

# Generate x and y values
x_values = np.linspace(0, 7.8254, 8)  # Adjusted to match the number of points in the original code
y_values = np.linspace(0, 7.8254, 8)  # Adjusted to match the number of points in the original code

# Generate all combinations of x and y coordinates
coordinates = [(x, y) for x in x_values for y in y_values]

# Create a colormap plot based on SCFshift values
plt.imshow(scf_shift_values.values.reshape(8, 8), cmap='viridis', extent=[0, 7.8254, 0, 7.8254], origin='lower', interpolation='bicubic')

# Add cluster numbers as text annotations at the center of each coordinate
for i, coord in enumerate(coordinates):
    if cluster_labels[i] == 0:
        plt.text(coord[0], coord[1], str(cluster_labels[i]), ha='center', va='center', color='blue')
    elif cluster_labels[i] == 1:
        plt.text(coord[0], coord[1], str(cluster_labels[i]), ha='center', va='center', color='white')
    elif cluster_labels[i] == 2:
        plt.text(coord[0], coord[1], str(cluster_labels[i]), ha='center', va='center', color='red')
    elif cluster_labels[i] == 3:
        plt.text(coord[0], coord[1], str(cluster_labels[i]), ha='center', va='center', color='yellow')

# Remove x and y axis
plt.axis('off')

colorbar = plt.colorbar()
colorbar.ax.set_ylabel('SCF Shift (Ry)')

# Adjust the spacing between the plot and the title
plt.subplots_adjust(top=0.9)

# Display the plot
plt.show()

### Silhouette

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Import the data from CSV
data = pd.read_csv('SCFshift.csv', header=None)
SCFshift = data.iloc[:, 0].values

# Check the number of unique values
unique_values = np.unique(SCFshift)
num_unique = len(unique_values)

if num_unique < 2:
    print("Error: Number of unique values is less than 2. Check your data.")
    exit()

# Perform clustering and calculate silhouette scores
silhouette_scores = []
clusters = range(2, min(num_unique, 15))

for n_clusters in clusters:
    # Fit the K-means model
    kmeans = KMeans(max_iter=100, n_clusters=n_clusters)
    kmeans.fit(SCFshift.reshape(-1, 1))

    # Predict cluster labels
    labels = kmeans.predict(SCFshift.reshape(-1, 1))

    # Calculate silhouette score
    score = silhouette_score(SCFshift.reshape(-1, 1), labels)
    silhouette_scores.append(score)

# Plot silhouette scores
plt.plot(clusters, silhouette_scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

## Polynomial Regression

In [ ]:
data_for_regression = data.copy()

In [ ]:
rows = [0, 0, 0, 0]

data['index'] = range(0, len(data))

for i in range(0, 63):
    if data_for_regression['Cluster_labels'][i] == 0 and rows[0] == 0:
        rows[0] = data_for_regression['index'][i]
    elif data['Cluster_labels'][i] == 1 and rows[1] == 0:
        rows[1] = data_for_regression['index'][i]
    elif data['Cluster_labels'][i] == 2 and rows[2] == 0:
        rows[2] = data_for_regression['index'][i]
    elif data['Cluster_labels'][i] == 3 and rows[3] == 0:
        rows[3] = data_for_regression['index'][i]

# make rows a 1D array
rows = np.array(rows)

# Input top features | Ex. top_features = ['G4-3  R_cut = 8', 'G5-5  R_cut = 11']
indiv_feature_ary = []

selected_feature_df = data[top_features_selected]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

r2_scores = []
degrees = []

for i in range(1, 10):
    # Take specific rows of the DataFrame for the training set
    selected_vars = selected_feature_df.iloc[rows]
    selected = data_for_regression.iloc[rows]

    # Create DataFrames with the rest of the rows for the testing set
    unselected_vars = data_for_regression[indiv_feature_ary].drop(selected_vars.index)
    unselected = data_for_regression.drop(selected.index)

    # Create polynomial features for the training and testing sets
    degree = i  # You can adjust the degree as needed
    poly_features = PolynomialFeatures(degree=degree)
    X_train = poly_features.fit_transform(selected_vars)
    X_test = poly_features.transform(unselected_vars)

    # Prepare the target variables for training and testing sets
    y_train = selected["SCFshift"].values
    y_test = unselected["SCFshift"].values

    # Create and train the linear regression model
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    # Make predictions on the testing set
    y_pred = regressor.predict(X_test)

    # Evaluate the model using R-squared (R²) score
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    degrees.append(degree)

# plot r2 on the y and degrees on the x
plt.plot(degrees, r2_scores)
plt.xlabel('Degree')
plt.ylabel('R2 Score')
plt.show()
